#EDA

###Importaciones

In [2]:
import pandas as pd
import json
import numpy as np
import pickle
import pyarrow.parquet as pq
import os

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Funciones de normalizacion y carga

In [ ]:
#la función se encarga de limpiar espacios en blanco alrededor de cada categoría en la lista, pero si la lista original es nula, devuelve None
def normalize_categories(category_list):
    if category_list is not None:
        return [category.strip() for category in category_list]
    else:
        return None

In [ ]:
#la función se encarga de procesar y normalizar las horas en el formato "día: horas" para cada entrada de día y horas en la lista original. Si la lista original es nula, la función devuelve None.
def normalize_hours(hours_list):
    if hours_list is not None:
        normalized_hours = []
        for day_hours in hours_list:
            if len(day_hours) == 2:
                day, hours = day_hours
                normalized_day_hours = f"{day}: {hours}"
                normalized_hours.append(normalized_day_hours)
        return normalized_hours
    else:
        return None

In [ ]:
#la función se encarga de "aplanar" un diccionario, convirtiendo las listas asociadas a las claves en entradas adicionales en el nuevo diccionario, donde cada elemento de la lista se concatena al nombre de la clave original
def flatten_dict(d):
    if d is None:
        return {}

    flattened_dict = {}
    for key, values in d.items():
        if isinstance(values, list):
            for value in values:
                flattened_dict[f"{key}: {value}"] = 1
        else:
            flattened_dict[key] = values
    return flattened_dict


In [ ]:
#la función carga datos desde un archivo JSON, normaliza las categorías y las horas de operación en el DataFrame, y devuelve el DataFrame resultante
def cargar_y_normalizar(ruta_archivo):
    with open(ruta_archivo, 'r') as f:
        j = [json.loads(line) for line in f]
    df = pd.DataFrame(j)
    df['category'] = df['category'].apply(normalize_categories)
    df_meta = df.dropna(subset=['category'])
    df_meta = df_meta.explode('category')
    df_meta = df_meta.drop(columns=['relative_results'])
    df_meta = df_meta.drop(columns=['MISC'])
    df_meta['hours'] = df_meta['hours'].apply(normalize_hours)
    df_meta['hours'] = df_meta['hours'].apply(lambda x: ', '.join(x) if x is not None else None)
    return df_meta

### Apertura y normalizacion de archivos de MetaData

In [ ]:
ruta_archivo1 = '/content/drive/MyDrive/Data/Google Maps/metadata-sitios/1.json'
ruta_archivo2 = '/content/drive/MyDrive/Data/Google Maps/metadata-sitios/2.json'
ruta_archivo3 = '/content/drive/MyDrive/Data/Google Maps/metadata-sitios/3.json'
ruta_archivo4 = '/content/drive/MyDrive/Data/Google Maps/metadata-sitios/4.json'
ruta_archivo5 = '/content/drive/MyDrive/Data/Google Maps/metadata-sitios/5.json'
ruta_archivo6 = '/content/drive/MyDrive/Data/Google Maps/metadata-sitios/6.json'
ruta_archivo7 = '/content/drive/MyDrive/Data/Google Maps/metadata-sitios/7.json'
ruta_archivo8 = '/content/drive/MyDrive/Data/Google Maps/metadata-sitios/8.json'
ruta_archivo9 = '/content/drive/MyDrive/Data/Google Maps/metadata-sitios/9.json'
ruta_archivo10 = '/content/drive/MyDrive/Data/Google Maps/metadata-sitios/10.json'
ruta_archivo11 = '/content/drive/MyDrive/Data/Google Maps/metadata-sitios/11.json'



df_meta1 = cargar_y_normalizar(ruta_archivo1)
df_meta2 = cargar_y_normalizar(ruta_archivo2)
df_meta3 = cargar_y_normalizar(ruta_archivo3)
df_meta4 = cargar_y_normalizar(ruta_archivo4)
df_meta5 = cargar_y_normalizar(ruta_archivo5)
df_meta6 = cargar_y_normalizar(ruta_archivo6)
df_meta7 = cargar_y_normalizar(ruta_archivo7)
df_meta8 = cargar_y_normalizar(ruta_archivo8)
df_meta9 = cargar_y_normalizar(ruta_archivo9)
df_meta10 = cargar_y_normalizar(ruta_archivo10)
df_meta11 = cargar_y_normalizar(ruta_archivo11)

Una vez cargados los archivos procedemos a filtrar solo las categorias que contengan la palabra "Hotel"

In [ ]:
categoria_filtrado= 'Hotel'

In [ ]:
df_meta_n1 = df_meta1[df_meta1['category'].str.contains(categoria_filtrado, case=False, na=False)]
df_meta_n2 = df_meta2[df_meta2['category'].str.contains(categoria_filtrado, case=False, na=False)]
df_meta_n3 = df_meta3[df_meta3['category'].str.contains(categoria_filtrado, case=False, na=False)]
df_meta_n4 = df_meta4[df_meta4['category'].str.contains(categoria_filtrado, case=False, na=False)]
df_meta_n5 = df_meta5[df_meta5['category'].str.contains(categoria_filtrado, case=False, na=False)]
df_meta_n6 = df_meta6[df_meta6['category'].str.contains(categoria_filtrado, case=False, na=False)]
df_meta_n7 = df_meta7[df_meta7['category'].str.contains(categoria_filtrado, case=False, na=False)]
df_meta_n8 = df_meta8[df_meta8['category'].str.contains(categoria_filtrado, case=False, na=False)]
df_meta_n9 = df_meta9[df_meta9['category'].str.contains(categoria_filtrado, case=False, na=False)]
df_meta_n10 = df_meta10[df_meta10['category'].str.contains(categoria_filtrado, case=False, na=False)]
df_meta_n11 = df_meta11[df_meta11['category'].str.contains(categoria_filtrado, case=False, na=False)]

#se concatena toda la meta data
df_total_meta = pd.concat([df_meta_n1, df_meta_n2, df_meta_n3, df_meta_n4, df_meta_n5, df_meta_n6, df_meta_n7, df_meta_n8, df_meta_n9, df_meta_n10, df_meta_n11], ignore_index=True)

#se guarda el MetaData en CSV con la categoria seleccionada
df_total_meta.to_csv('/content/drive/My Drive/Colab Notebooks/metadata.csv', index=False)

procedemos a guardar la variable gmap_id para luego utilizarla y solo guardar los reviews que necesito de los estados

In [ ]:
df_gmap_id = df_total_meta['gmap_id']

Abro los archivos de review de los 5 estados que vamos a utilizar y los guardo en un solo DataFrame

In [ ]:
# Lista de estados a procesar  ['California', 'Washington', 'Oregon', 'Nevada', 'Arizona']
estados = ['Arizona']

In [ ]:
def procesar_estado(estado):
    directorio = f'/content/drive/MyDrive/Data/Google Maps/reviews-estados/review-{estado}'
    df_estado = pd.DataFrame()

    def limpiar_datos(line):
        try:
            # Reemplazar caracteres de escape por espacios en blanco
            line = line.replace('\\', ' ')
            return json.loads(line)
        except json.JSONDecodeError:
            return None

    for archivo_json in os.listdir(directorio):
        if archivo_json.endswith('.json'):
            ruta_completa = os.path.join(directorio, archivo_json)
            with open(ruta_completa, 'r', encoding='utf-8') as f:
                datos = [limpiar_datos(line) for line in f if line.strip()]
                datos = [d for d in datos if d is not None]  # Eliminar datos nulos
                df_temp = pd.DataFrame(datos)

                if 'pics' in df_temp.columns:
                    df_temp.drop('pics', axis=1, inplace=True)
                if 'resp' in df_temp.columns:
                    df_temp.drop('resp', axis=1, inplace=True)
                # Rellenar valores nulos con ceros
                df_temp.fillna(0, inplace=True)

                df_estado = pd.concat([df_estado, df_temp], ignore_index=True)

    # Del estado, solo busco y concateno los gmap_id que están en MetaData
    df_resultado = df_estado.merge(df_gmap_id, on='gmap_id', how='inner')

    # Guardar el archivo en CSV del estado
    df_resultado.to_csv(f'/content/drive/My Drive/Colab Notebooks/{estado.lower()}_estado.csv', index=False)

In [ ]:
# Procesar cada estado
for estado in estados:
    procesar_estado(estado)

una vez filtrados los estados de la zona oeste de Estados Unidos con la categoria hoteles, procedemos a realizar un solo DataFrame de toda la informacion extraida

abrimos todos los archivos

In [3]:
directorio1 = f'/content/drive/MyDrive/Colab Notebooks/MetaData-Estados/california_estado.csv'
directorio2 = f'/content/drive/MyDrive/Colab Notebooks/MetaData-Estados/arizona_estado.csv'
directorio3 = f'/content/drive/MyDrive/Colab Notebooks/MetaData-Estados/nevada_estado.csv'
directorio4 = f'/content/drive/MyDrive/Colab Notebooks/MetaData-Estados/oregon_estado.csv'
directorio5 = f'/content/drive/MyDrive/Colab Notebooks/MetaData-Estados/washington_estado.csv'
metadata = f'/content/drive/MyDrive/Colab Notebooks/MetaData-Estados/metadata.csv'

california = pd.read_csv(directorio1)
arizona = pd.read_csv(directorio2)
nevada = pd.read_csv(directorio3)
oregon = pd.read_csv(directorio4)
washington = pd.read_csv(directorio5)
metadata = pd.read_csv(metadata)

In [4]:
metadata.head(2)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,state,url
0,Basecamp Guesthouse,"Basecamp Guesthouse, 49010 SE Middle Fork Rd, ...",0x54907fca2751e187:0x3d6ae31ac18da483,NaN,47.471332,-121.686928,Hotel,5.0,8,NaN,NaN,NaN,https://www.google.com/maps/place//data=!4m2!3...
1,Sugar River Loft,"Sugar River Loft, 929 W Exchange St, Brodhead,...",0x88062b28cb53ef6d:0x94df72568fddf391,NaN,42.619862,-89.377890,Hotel,4.7,8,NaN,NaN,NaN,https://www.google.com/maps/place//data=!4m2!3...


In [7]:
# Cambiar el nombre de la columna
metadata.rename(columns={'name': 'name_hotel'}, inplace=True)

In [5]:
california.head(2)

,user_id,name,time,rating,text,gmap_id
0,108231792947173216700,TCSkyline,1618337529038,5,My wife and I came to ojai for our anniversary...,0x80e9bab58b7fb435:0xb3ab3c94d85f3b5c
1,115315553077382023272,Melanie Galuten,1551289391212,5,Extraordinary massage and facial!!! Have had...,0x80e9bab58b7fb435:0xb3ab3c94d85f3b5c


In [9]:
# Lista de directorios de los archivos CSV de estados
directorios_estados = [
    '/content/drive/MyDrive/Colab Notebooks/MetaData-Estados/california_estado.csv',
    '/content/drive/MyDrive/Colab Notebooks/MetaData-Estados/arizona_estado.csv',
    '/content/drive/MyDrive/Colab Notebooks/MetaData-Estados/nevada_estado.csv',
    '/content/drive/MyDrive/Colab Notebooks/MetaData-Estados/oregon_estado.csv',
    '/content/drive/MyDrive/Colab Notebooks/MetaData-Estados/washington_estado.csv'
]

# Leer los archivos CSV y almacenarlos en una lista
estados = [pd.read_csv(directorio) for directorio in directorios_estados]

# Combinar las tablas de cada estado con la tabla de metadata
combined_tables = [pd.merge(estado, metadata, on='gmap_id', how='inner') for estado in estados]

# Seleccionar las columnas deseadas y renombrarlas
result_tables = [
    table[['user_id', 'name', 'time', 'rating', 'text', 'gmap_id','name_hotel', 'address', 'description', 'latitude', 'longitude', 'category', 'avg_rating', 'num_of_reviews', 'price', 'hours', 'state', 'url']]
    for table in combined_tables
]
# Renombrar las columnas
result_tables = [
    table.rename(columns={'name_x': 'name'})
    for table in result_tables
]

# Concatenar todas las tablas en una sola
final_table = pd.concat(result_tables, ignore_index=True)

# Mostrar las primeras filas de la tabla combinada
final_table.head(2)


,user_id,name,time,rating,text,gmap_id,name_hotel,address,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,state,url
0,108231792947173216700,TCSkyline,1618337529038,5,My wife and I came to ojai for our anniversary...,0x80e9bab58b7fb435:0xb3ab3c94d85f3b5c,The Day Spa of Ojai,"The Day Spa of Ojai, 209 N Montgomery St, Ojai...",NaN,34.449451,-119.243305,Resort hotel,4.8,28,NaN,NaN,NaN,https://www.google.com/maps/place//data=!4m2!3...
1,115315553077382023272,Melanie Galuten,1551289391212,5,Extraordinary massage and facial!!! Have had...,0x80e9bab58b7fb435:0xb3ab3c94d85f3b5c,The Day Spa of Ojai,"The Day Spa of Ojai, 209 N Montgomery St, Ojai...",NaN,34.449451,-119.243305,Resort hotel,4.8,28,NaN,NaN,NaN,https://www.google.com/maps/place//data=!4m2!3...


cambiamos el nombre de las columnas que se refieren a estrellas

In [10]:
final_table.rename(columns={'rating': 'review_stars'}, inplace=True)


In [11]:
final_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16469 entries, 0 to 16468
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   user_id         16469 non-null  object 
 1   name            16469 non-null  object 
 2   time            16469 non-null  int64  
 3   review_stars    16469 non-null  int64  
 4   text            16469 non-null  object 
 5   gmap_id         16469 non-null  object 
 6   name_hotel      16469 non-null  object 
 7   address         16389 non-null  object 
 8   description     8424 non-null   object 
 9   latitude        16469 non-null  float64
 10  longitude       16469 non-null  float64
 11  category        16469 non-null  object 
 12  avg_rating      16469 non-null  float64
 13  num_of_reviews  16469 non-null  int64  
 14  price           0 non-null      float64
 15  hours           0 non-null      object 
 16  state           0 non-null      object 
 17  url             16469 non-null 

eliminamos las columnas que no contienen datos

In [12]:
final_table.drop(['price', 'hours', 'state','avg_rating','name'], axis=1, inplace=True)

guardamos la tabla, previo al analisis de sentimiento a las reviews

In [13]:
final_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16469 entries, 0 to 16468
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   user_id         16469 non-null  object 
 1   time            16469 non-null  int64  
 2   review_stars    16469 non-null  int64  
 3   text            16469 non-null  object 
 4   gmap_id         16469 non-null  object 
 5   name_hotel      16469 non-null  object 
 6   address         16389 non-null  object 
 7   description     8424 non-null   object 
 8   latitude        16469 non-null  float64
 9   longitude       16469 non-null  float64
 10  category        16469 non-null  object 
 11  num_of_reviews  16469 non-null  int64  
 12  url             16469 non-null  object 
dtypes: float64(2), int64(3), object(8)
memory usage: 1.6+ MB


In [14]:
final_table.to_csv(f'/content/drive/My Drive/Colab Notebooks/MetaData-Estados.csv', index=False)

una vez guardado, comenzamos a realizar el analisis de sentimiento

In [15]:
from textblob import TextBlob
# Función para asignar categorías personalizadas
def categorizar_sentimiento(polaridad):
    if polaridad > 0.5:
        return 'Excelente'
    elif 0.1 <= polaridad <= 0.5:
        return 'Bueno'
    elif -0.1 <= polaridad < 0.1:
        return 'Regular'
    elif -0.5 <= polaridad < -0.1:
        return 'Malo'
    else:
        return 'Muy Malo'

# Aplica el análisis de sentimiento solo para reseñas presentes
mask = final_table['text'].notnull()
final_table.loc[mask, 'sentiment_analysis'] = final_table.loc[mask, 'text'].apply(lambda x: categorizar_sentimiento(TextBlob(str(x)).sentiment.polarity))

final_table.head(2)

,user_id,time,review_stars,text,gmap_id,name_hotel,address,description,latitude,longitude,category,num_of_reviews,url,sentiment_analysis
0,108231792947173216700,1618337529038,5,My wife and I came to ojai for our anniversary...,0x80e9bab58b7fb435:0xb3ab3c94d85f3b5c,The Day Spa of Ojai,"The Day Spa of Ojai, 209 N Montgomery St, Ojai...",NaN,34.449451,-119.243305,Resort hotel,28,https://www.google.com/maps/place//data=!4m2!3...,Excelente
1,115315553077382023272,1551289391212,5,Extraordinary massage and facial!!! Have had...,0x80e9bab58b7fb435:0xb3ab3c94d85f3b5c,The Day Spa of Ojai,"The Day Spa of Ojai, 209 N Montgomery St, Ojai...",NaN,34.449451,-119.243305,Resort hotel,28,https://www.google.com/maps/place//data=!4m2!3...,Excelente


Categorizamos la columna "review_stars" asignando las etiquetas de 1 como "Muy Malo", 2 como "Malo", 3 como "Regular", y 4 y 5 como "Bueno" y "Excelente", respectivamente. Posteriormente, comparamos esta clasificación con nuestro análisis de sentimiento, donde aplicamos también una categorización. En este caso, asignamos las siguientes etiquetas: 0 representa un comentario negativo, 1 un comentario neutro y 2 un comentario positivo.

In [16]:
# Define las categorías para las estrellas
star_mapping = {1: 'Muy Malo', 2: 'Malo', 3: 'Regular', 4: 'Bueno', 5: 'Excelente'}
final_table['stars_category'] = final_table['review_stars'].map(star_mapping)

In [17]:
# Selecciona las columnas que deseas mostrar
selected_columns = ['stars_category', 'sentiment_analysis','review_stars','text']

# Muestra las primeras 20 filas de las columnas seleccionadas
final_table[selected_columns].head(5)

,stars_category,sentiment_analysis,review_stars,text
0,Excelente,Excelente,5,My wife and I came to ojai for our anniversary...
1,Excelente,Excelente,5,Extraordinary massage and facial!!! Have had...
2,Excelente,Bueno,5,Love this place ud83d ude4f u2764 ud83e udd17
3,Excelente,Bueno,5,This was the best massage I u2019ve ever had! ...
4,Excelente,Excelente,5,Fritz was really good. I had a 90 min session ...


Como se observan rápidamente algunas discrepancias entre los dos análisis, creamos una tabla de contingencias para evaluarlas.

In [18]:
# Crea la tabla de contingencia
contingency_table = pd.crosstab(final_table['sentiment_analysis'], final_table['stars_category'], margins=True, margins_name='Total')

# Renombra la columna 'All' a 'Total'
contingency_table.rename(columns={'All': 'Total'}, inplace=True)

# Visualiza la tabla de contingencia
print(contingency_table)

stars_category      Bueno  Excelente  Malo  Muy Malo  Regular  Total
sentiment_analysis                                                  
Bueno                1450       2173   253       181      846   4903
Excelente             655       1984    26        26      194   2885
Malo                   39         69   163       388      106    765
Muy Malo               13          6    40       169       24    252
Regular              1555       3180   650       957     1322   7664
Total                3712       7412  1132      1721     2492  16469


se realiza un mapeo de las categorías de la columna "sentiment_analysis" a números del 1 al 5, asignándoles un valor numérico acorde a su grado de positividad. Luego, se asignan pesos a las métricas, especificando que la métrica de las estrellas de la revisión tiene un peso del 70%, mientras que el análisis de sentimiento tiene un peso del 30%. Finalmente, se calcula una nueva métrica combinada, donde se promedian ponderadamente los valores de las métricas de las estrellas y el análisis de sentimiento, obteniendo así una medida combinada que refleja tanto la evaluación de los usuarios como el análisis de sentimiento, considerando sus respectivos pesos. Este enfoque permite integrar ambas fuentes de información de manera equilibrada en una sola métrica para evaluar la satisfacción general

In [19]:
# Mapeo de categorías a números del 1 al 5
sentiment_mapping_numerico = {'Excelente': 5, 'Bueno': 4, 'Regular': 3, 'Malo': 2, 'Muy Malo': 1}

# Aplicar el mapeo a la columna sentiment_analysis
final_table['sentiment_analysis_numerico'] = final_table['sentiment_analysis'].map(sentiment_mapping_numerico)

# Asignar pesos a las métricas
peso_review_stars = 0.7
peso_analisis_sentimiento = 0.3

# Calcular métrica combinada
final_table['metrica_combinada'] = (peso_review_stars * final_table['review_stars'] +
                           peso_analisis_sentimiento * final_table['sentiment_analysis_numerico']) / (peso_review_stars + peso_analisis_sentimiento)

In [20]:
selected_columns = ['stars_category', 'sentiment_analysis','review_stars','metrica_combinada']

final_table[selected_columns].head(5)

,stars_category,sentiment_analysis,review_stars,metrica_combinada
0,Excelente,Excelente,5,5.0
1,Excelente,Excelente,5,5.0
2,Excelente,Bueno,5,4.7
3,Excelente,Bueno,5,4.7
4,Excelente,Excelente,5,5.0


In [25]:
final_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16469 entries, 0 to 16468
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   user_id            16469 non-null  object 
 1   date               16469 non-null  int64  
 2   gmap_id            16469 non-null  object 
 3   name               16469 non-null  object 
 4   address            16389 non-null  object 
 5   latitude           16469 non-null  float64
 6   longitude          16469 non-null  float64
 7   category           16469 non-null  object 
 8   num_of_reviews     16469 non-null  int64  
 9   url                16469 non-null  object 
 10  metrica_combinada  16469 non-null  float64
dtypes: float64(3), int64(2), object(6)
memory usage: 1.4+ MB


In [22]:
# eliminamos columnas que no vamos a utilizar
columnas_a_eliminar = ['sentiment_analysis_numerico', 'stars_category', 'sentiment_analysis','text','review_stars','description']

final_table.drop(columnas_a_eliminar, axis=1, inplace=True)

In [23]:
nombres_nuevos = {'time': 'date'}

final_table.rename(columns=nombres_nuevos, inplace=True)

In [24]:
nombres_nuevos = {'name_hotel': 'name'}

final_table.rename(columns=nombres_nuevos, inplace=True)

la columna "date" parecen ser representaciones de fechas en formato de tiempo UNIX, que es el número de milisegundos transcurridos desde la medianoche UTC del 1 de enero de 1970. se procede a realizar una transformacion en estos

In [27]:
# Convertir la columna 'date' de milisegundos a segundos y luego a formato de fecha
final_table['date'] = pd.to_datetime(final_table['date'] // 1000, unit='s')

# Mostrar las primeras filas de la columna 'date'
final_table['date'].head()

0   2021-04-13 18:12:09
1   2019-02-27 17:43:11
2   2021-05-27 16:48:51
3   2020-02-03 04:39:51
4   2017-07-08 22:00:48
Name: date, dtype: datetime64[ns]

In [28]:
final_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16469 entries, 0 to 16468
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   user_id            16469 non-null  object        
 1   date               16469 non-null  datetime64[ns]
 2   gmap_id            16469 non-null  object        
 3   name               16469 non-null  object        
 4   address            16389 non-null  object        
 5   latitude           16469 non-null  float64       
 6   longitude          16469 non-null  float64       
 7   category           16469 non-null  object        
 8   num_of_reviews     16469 non-null  int64         
 9   url                16469 non-null  object        
 10  metrica_combinada  16469 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(6)
memory usage: 1.4+ MB


guardamos como csv con analisis de sentimientos realizado.

In [29]:
final_table.to_csv('/content/drive/My Drive/Colab Notebooks/MetaData-Estados-Sentiment.csv', index=False)

In [ ]:
final_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16469 entries, 0 to 16468
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   user_id                      16469 non-null  object 
 1   name                         16469 non-null  object 
 2   time                         16469 non-null  int64  
 3   review_stars                 16469 non-null  int64  
 4   text                         16469 non-null  object 
 5   gmap_id                      16469 non-null  object 
 6   address                      16389 non-null  object 
 7   description                  8424 non-null   object 
 8   latitude                     16469 non-null  float64
 9   longitude                    16469 non-null  float64
 10  category                     16469 non-null  object 
 11  num_of_reviews               16469 non-null  int64  
 12  url                          16469 non-null  object 
 13  sentiment_analys

vamos a realizar una agrupacion de la tabla con nuestros reviews y nuestra metrica_combinada para llegar a una tabla final con negocion unicos y un analisis de sentimiento realizado sobre las reviews a estos negocios

In [ ]:
# Seleccionar las columnas deseadas
selected_columns = ['user_id', 'name', 'gmap_id', 'address', 'description', 'latitude', 'longitude', 'num_of_reviews', 'url', 'category', 'metrica_combinada']

# Agrupar por 'gmap_id' y calcular el promedio de 'metrica_combinada'
df_grouped_business = final_table[selected_columns].groupby('gmap_id', as_index=False).agg({
    'metrica_combinada': 'mean',
    'address': 'first',  # Tomar el primer valor para 'address'
    'description': 'first',  # Tomar el primer valor para 'description'
    'latitude': 'first',  # Tomar el primer valor para 'latitude'
    'longitude': 'first',  # Tomar el primer valor para 'longitude'
    'num_of_reviews': 'first',  # Tomar el primer valor para 'num_of_reviews'
    'url': 'first',  # Tomar el primer valor para 'url'
    'category': 'first'  # Tomar el primer valor para 'category'
})

# Renombrar la columna 'metrica_combinada' a 'stars'
df_grouped_business = df_grouped_business.rename(columns={'metrica_combinada': 'stars'})

# Visualizar el nuevo DataFrame agrupado
df_grouped_business.head(2)



,gmap_id,stars,address,description,latitude,longitude,num_of_reviews,url,category
0,0x5361e635a3a56823:0xb26d337a3f0e01a9,4.178261,"Newman Lake Resort & Marina, 12515 NE Newman L...",None,47.771818,-117.081735,38,https://www.google.com/maps/place//data=!4m2!3...,Resort hotel
1,0x53623b3cc7cec4d9:0xa4e1cfecb43edddf,4.257831,"Marshall Lake Resort, 1301 Marshall Lake Rd, N...",None,48.256318,-117.079852,133,https://www.google.com/maps/place//data=!4m2!3...,Resort hotel


In [ ]:
df_grouped_business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294 entries, 0 to 293
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   gmap_id         294 non-null    object 
 1   stars           294 non-null    float64
 2   address         293 non-null    object 
 3   description     121 non-null    object 
 4   latitude        294 non-null    float64
 5   longitude       294 non-null    float64
 6   num_of_reviews  294 non-null    int64  
 7   url             294 non-null    object 
 8   category        294 non-null    object 
dtypes: float64(3), int64(1), object(5)
memory usage: 20.8+ KB


guardamos el dataframe

In [ ]:
df_grouped_business.to_csv('/content/drive/My Drive/Colab Notebooks/MetaData-Estados-Final.csv', index=False)

### Yelp

se comienza con el ETL de los archivos de YELP

In [ ]:
archivo_pkl = '/content/drive/MyDrive/Data/Yelp/business.pkl'
with open(archivo_pkl, 'rb') as archivo:
    business = pickle.load(archivo)
business = business.loc[:, ~business.columns.duplicated()]
business = business.drop(columns=['attributes'])
business = business.drop(columns=['hours'])

Selecciono al igual que en MetaData las categorias de yelp que vamos a utilizar

In [ ]:
categoria_especifica_yelp = 'Hotel'

In [ ]:
business = business[business['categories'].str.contains(categoria_especifica_yelp, case=False, na=False)]

In [ ]:
business.head(2)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
18,8wGISYjYkE2tSqn3cDMu8A,Nifty Car Rental,1241 Airline Dr,Kenner,PA,70062,29.981183,-90.254012,3.5,14,1,"Automotive, Car Rental, Hotels & Travel, Truck..."
34,w_AMNoI1iG9eay7ncmc67w,River 127,100 Iberville St,New Orleans,PA,70130,29.951359,-90.064672,3.0,12,1,"Event Planning & Services, Hotels, Hotels & Tr..."


al realizar ciertas pruebas de los datos, descubrimo que las columnas state y city contienen datos errones, debito a esto procedemos a utilizar la libreria reverse_geocoder para a traves de la longitud y la latitud poder averiguar la ciudad y el estado correcto de los negocios

In [ ]:
import reverse_geocoder as rg

def obtener_estado_y_ciudad(lat, lon):
    coordinates = (lat, lon)
    result = rg.search(coordinates)  # Realiza la búsqueda inversa
    estado = result[0]['admin1']  # Obtiene el estado de la respuesta
    ciudad = result[0]['name']  # Obtiene el nombre de la ciudad de la respuesta
    return estado, ciudad
def obtener_estado_y_ciudad(lat, lon):
    coordinates = (lat, lon)
    result = rg.search(coordinates)  # Realiza la búsqueda inversa
    estado = result[0]['admin1']  # Obtiene el estado de la respuesta
    ciudad = result[0]['name']  # Obtiene el nombre de la ciudad de la respuesta
    return estado, ciudad

# Aplica la función a cada fila del DataFrame
business[['estado', 'ciudad']] = business.apply(lambda x: pd.Series(obtener_estado_y_ciudad(x['latitude'], x['longitude'])), axis=1)

Loading formatted geocoded file...


In [ ]:
business.head(20)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,estado,ciudad
18,8wGISYjYkE2tSqn3cDMu8A,Nifty Car Rental,1241 Airline Dr,Kenner,PA,70062,29.981183,-90.254012,3.5,14,1,"Automotive, Car Rental, Hotels & Travel, Truck...",Louisiana,Kenner
34,w_AMNoI1iG9eay7ncmc67w,River 127,100 Iberville St,New Orleans,PA,70130,29.951359,-90.064672,3.0,12,1,"Event Planning & Services, Hotels, Hotels & Tr...",Louisiana,New Orleans
55,xM6LoUcnpDpMBzXs_7dXAg,Fairfield Inn & Suites,719 E Baltimore Pike,Kennett Square,AB,19348,39.856248,-75.69461,3.0,37,1,"Hotels, Hotels & Travel, Event Planning & Serv...",Pennsylvania,Kennett Square
65,uczmbBk5O3tYhGue13dCDg,New Orleans Spirit Tours,723 St Peter St,New Orleans,IN,70130,29.958431,-90.065173,4.0,38,1,"Hotels & Travel, Tours, Local Flavor",Louisiana,New Orleans
67,eYxGFkxo6m3SYGVTh5m2nQ,Big Boyz Toyz Motorcycle Rentals,4158 E Grant Rd,Tucson,PA,85712,32.250324,-110.903655,4.5,8,1,"Towing, Hotels & Travel, Automotive, Motorcycl...",Arizona,Tucson
112,sB45WFgysT617bKWP_WJwA,Budweiser Brewery Experience,1200 Lynch St,Saint Louis,PA,63118,38.600197,-90.213538,4.5,605,1,"Bars, Beer Gardens, Food, Breweries, Nightlife...",Missouri,St. Louis
143,-aeZuatjCDMV1X4gCTz9Ug,David Thomas Trailways,14005 McNulty Rd,Philadelphia,FL,19154,40.106409,-74.973937,4.5,6,1,"Buses, Transportation, Bus Tours, Hotels & Tra...",Pennsylvania,Trevose
155,hUQ9Z7kQeabvhPOAQOVV1A,Rathbone Mansions,1244 Esplanade Ave,New Orleans,IN,70116,29.967055,-90.065828,3.5,67,1,"Hotels, Hotels & Travel, Bed & Breakfast, Even...",Louisiana,New Orleans
181,ORL4JE6tz3rJxVqkdKfegA,Gaylord Opryland Resort & Convention Center,2800 Opryland Dr,Nashville,NV,37214,36.211592,-86.694319,3.0,1639,1,"Venues & Event Spaces, Performing Arts, Arts &...",Tennessee,Lakewood
197,znL4YrNbA2uwOQ3CchkmEA,Seaview Condominiums,14700 Gulf Blvd,Madeira Beach,PA,33708,27.798503,-82.79875,4.5,8,1,"Home Services, Vacation Rentals, Real Estate, ...",Florida,Madeira Beach


In [ ]:
business.rename(columns={'stars': 'business_stars'}, inplace=True)

In [ ]:
business.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5858 entries, 18 to 150331
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   business_id     5858 non-null   object
 1   name            5858 non-null   object
 2   address         5858 non-null   object
 3   city            5858 non-null   object
 4   state           5858 non-null   object
 5   postal_code     5858 non-null   object
 6   latitude        5858 non-null   object
 7   longitude       5858 non-null   object
 8   business_stars  5858 non-null   object
 9   review_count    5858 non-null   object
 10  is_open         5858 non-null   object
 11  categories      5858 non-null   object
 12  estado          5858 non-null   object
 13  ciudad          5858 non-null   object
dtypes: object(14)
memory usage: 686.5+ KB


filtro solo los estados a utilizar

luego de realizar una normalizacion de los estados y las ciudades para corregir los datos proporcionados que eran erroneos, al realizar una busqueda de todos los estados que disponiamos, allamos que solo contamos con negocios en California y Nevada enfocandonos en nuestra propuesta de trabajar con el sector oeste de Estados Unidos, procedemos a filtrar estos dos estados y en un segundo paso a traves de la Api de YELP a poblar nuestra base de datos con nuevas ciudades y estados.

In [ ]:
unicos = business["estado"].unique()
unicos

array(['Louisiana', 'Pennsylvania', 'Arizona', 'Missouri', 'Tennessee',
       'Florida', 'California', 'Nevada', 'Idaho', 'Alberta',
       'New Jersey', 'Indiana', 'Illinois', 'Delaware'], dtype=object)

In [ ]:
# Lista de estados a filtrar
estados_filtrar = ['California', 'Oregon', 'Washington', 'Arizona', 'Nevada']

# Filtrar el DataFrame para incluir solo los estados deseados
business = business[business['estado'].isin(estados_filtrar)]

In [ ]:
business.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 930 entries, 67 to 150331
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   business_id     930 non-null    object
 1   name            930 non-null    object
 2   address         930 non-null    object
 3   city            930 non-null    object
 4   state           930 non-null    object
 5   postal_code     930 non-null    object
 6   latitude        930 non-null    object
 7   longitude       930 non-null    object
 8   business_stars  930 non-null    object
 9   review_count    930 non-null    object
 10  is_open         930 non-null    object
 11  categories      930 non-null    object
 12  estado          930 non-null    object
 13  ciudad          930 non-null    object
dtypes: object(14)
memory usage: 109.0+ KB


guardamos el dataframe con nuestros hoteles filtrados por estados para su proximo uso

In [ ]:
business.to_csv('/content/drive/My Drive/Colab Notebooks/business-filtrado.csv', index=False, encoding='utf-8')

In [ ]:
#Abrimos el archivo checkin.json
with open ('/content/drive/MyDrive/Data/Yelp/checkin.json','r') as f:
  jcheckin = [json.loads(line) for line in f]

df_checkin_Yelp= pd.DataFrame(jcheckin)

In [ ]:
df_checkin_Yelp.head(2)

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."


se realiza un merge entre las dos tablas mediante Business_id y se guarda solo los datos que corresponden a la caterogia seleccionada

In [ ]:
combined_df = df_checkin_Yelp.merge(business, on='business_id', how='inner')

# creamos un CSV mas pequeño para el Machine Learning de Business y Checkin
columnas_seleccionadas_bc = ['business_id', 'date','state']
df_seleccionado_bc = combined_df[columnas_seleccionadas_bc]
df_seleccionado_bc.to_csv('/content/drive/My Drive/Colab Notebooks/Yelp Checkin Business.csv', index=False)

In [ ]:
df_seleccionado_bc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 803 entries, 0 to 802
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   business_id  803 non-null    object
 1   date         803 non-null    object
 2   state        803 non-null    object
dtypes: object(3)
memory usage: 25.1+ KB


In [ ]:
# Cuenta los valores nulos en la columna 'date'
nulos_date = df_seleccionado_bc['date'].isnull().sum()

# Muestra la cantidad de valores nulos
print(f"Cantidad de valores nulos en la columna 'date': {nulos_date}")


Cantidad de valores nulos en la columna 'date': 0


In [ ]:
df_seleccionado_bc.head(2)

,business_id,date,state
0,-12_gQ7NRcMWSRs97mRQdw,"2017-02-15 04:31:22, 2017-03-07 04:38:30, 2018...",CA
1,-7kN5J6yiT3sZbsiDHquPA,"2013-02-25 16:29:45, 2019-01-19 21:42:18",AZ


creamos una tabla de años y meses para contar la cantidad de checkins para todos los negocios que vamos utilizar y ademas poder tener una vision de el flujo de gente.

In [ ]:
from itertools import product

# Genera todas las combinaciones posibles de año y mes
years = range(2010, 2023)
months = range(1, 13)
all_combinations = list(product(years, months))

# Crea un DataFrame inicial con todas las combinaciones y la cantidad de fechas igual a cero
df_empty = pd.DataFrame(all_combinations, columns=['year', 'month'])
df_empty['count'] = 0

# Visualiza el DataFrame vacío
print(df_empty)

     year  month  count
0    2010      1      0
1    2010      2      0
2    2010      3      0
3    2010      4      0
4    2010      5      0
..    ...    ...    ...
151  2022      8      0
152  2022      9      0
153  2022     10      0
154  2022     11      0
155  2022     12      0

[156 rows x 3 columns]


recorremos nuestro dataframe combinado de business.pkl y checking.csv para obtener un dataframe con el flujo de gente para nuestros negocios de interes y con este obtener un tabla divida por año y mes con la cantidad de personas que realizaron un checking.

In [ ]:
from collections import Counter
from datetime import datetime
from itertools import product

# Crear un DataFrame vacío con todas las combinaciones posibles de año, mes y estado
years = range(2010, 2023)
months = range(1, 13)
states = df_seleccionado_bc['state'].unique()
all_combinations = list(product(years, months, states))
df_empty = pd.DataFrame(all_combinations, columns=['year', 'month', 'state'])
df_empty['count'] = 0

# Convertir la columna 'date' en una lista de listas de fechas
df_seleccionado_bc['date'] = df_seleccionado_bc['date'].apply(lambda x: [] if pd.isna(x) else x.split(", "))
df_seleccionado_bc['date'] = df_seleccionado_bc['date'].apply(lambda x: [datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S") for date_str in x])

# Contar la frecuencia de fechas por año, mes y estado
date_state_counter = Counter()
for _, row in df_seleccionado_bc.iterrows():
    for date in row['date']:
        date_state_counter[(date.year, date.month, row['state'])] += 1

# Actualizar las filas en el DataFrame df_empty
df_empty['count'] = df_empty.apply(lambda row: date_state_counter[(row['year'], row['month'], row['state'])], axis=1)

# Visualizar el resultado actualizado
df_empty

<ipython-input-26-8781ac6067f0>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_seleccionado_bc['date'] = df_seleccionado_bc['date'].apply(lambda x: [] if pd.isna(x) else x.split(", "))
<ipython-input-26-8781ac6067f0>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_seleccionado_bc['date'] = df_seleccionado_bc['date'].apply(lambda x: [datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S") for date_str in x])


,year,month,state,count
0,2010,1,CA,1
1,2010,1,AZ,6
2,2010,1,NV,2
3,2010,2,CA,3
4,2010,2,AZ,20
...,...,...,...,...
463,2022,11,AZ,0
464,2022,11,NV,0
465,2022,12,CA,0
466,2022,12,AZ,0


se procede a guardar la tabla en un archivo csv

In [ ]:
df_empty.to_csv('/content/drive/My Drive/Colab Notebooks/Tabla-Checkin-Ano-Mes.csv', index=False)

Abro el archivo Review.json y creo una funcion para limpiarlo

In [ ]:
review = '/content/drive/MyDrive/Data/Yelp/review-002.json'
rows_to_read = 5000000  # Número total de filas que deseas leer
chunk_size = 1000000  # Tamaño del fragmento

def limpiar_datos(line):
    try:
        line = line.replace('\\', ' ')
        return json.loads(line)
    except json.JSONDecodeError:
        return None

# Inicializar el DataFrame
df_review = pd.DataFrame()

# Crear un generador para cargar el archivo en fragmentos
chunks = pd.read_json(review, lines=True, chunksize=chunk_size, nrows=rows_to_read)

# Iterar sobre los fragmentos y procesar cada uno
for chunk in chunks:
    datos = [limpiar_datos(line) for line in chunk.to_json(orient='records', lines=True).split('\n') if line.strip()]
    datos = [d for d in datos if d is not None]  # Eliminar datos nulos
    df_temp = pd.DataFrame(datos)

    if 'user_id' in df_temp.columns:
        df_temp.drop('user_id', axis=1, inplace=True)
    if 'cool' in df_temp.columns:
        df_temp.drop('cool', axis=1, inplace=True)
    if 'funny' in df_temp.columns:
        df_temp.drop('funny', axis=1, inplace=True)
    if 'useful' in df_temp.columns:
        df_temp.drop('useful', axis=1, inplace=True)

    # Rellenar valores nulos con ceros
    df_temp.fillna(0, inplace=True)

    # Concatenar el fragmento al DataFrame principal
    df_review = pd.concat([df_review, df_temp], ignore_index=True)

    # Verificar si ya se han leído las filas deseadas
    if len(df_review) >= rows_to_read:
        break

In [ ]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4421848 entries, 0 to 4421847
Data columns (total 5 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   review_id    object
 1   business_id  object
 2   stars        int64 
 3   text         object
 4   date         int64 
dtypes: int64(2), object(3)
memory usage: 168.7+ MB


 la columna "date" parecen ser representaciones de fechas en formato de tiempo UNIX, que es el número de milisegundos transcurridos desde la medianoche UTC del 1 de enero de 1970. se procede a realizar una transformacion en estos

In [ ]:
df_review['date'].head()

0    1531001351000
1    1325604498000
2    1391632230000
3    1420329663000
4    1484427255000
Name: date, dtype: int64

In [ ]:
# Convertir la columna 'date' de milisegundos a segundos y luego a formato de fecha
df_review['date'] = pd.to_datetime(df_review['date'] // 1000, unit='s')

# Mostrar las primeras filas de la columna 'date'
df_review['date'].head()

0   2018-07-07 22:09:11
1   2012-01-03 15:28:18
2   2014-02-05 20:30:30
3   2015-01-04 00:01:03
4   2017-01-14 20:54:15
Name: date, dtype: datetime64[ns]

combinamos nuestros archivos review.json con busineess.pkl

In [ ]:
combined_df_business_reviwes = df_review.merge(estados, on='business_id', how='inner')


In [ ]:
combined_df_business_reviwes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47691 entries, 0 to 47690
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   review_id       47691 non-null  object        
 1   business_id     47691 non-null  object        
 2   stars           47691 non-null  int64         
 3   text            47691 non-null  object        
 4   date            47691 non-null  datetime64[ns]
 5   name            47691 non-null  object        
 6   address         46257 non-null  object        
 7   city            47691 non-null  object        
 8   state           47691 non-null  object        
 9   postal_code     47684 non-null  float64       
 10  latitude        47691 non-null  float64       
 11  longitude       47691 non-null  float64       
 12  business_stars  47691 non-null  float64       
 13  review_count    47691 non-null  int64         
 14  is_open         47691 non-null  int64         
 15  ca

eliminamos columnas que no vamos a utilizar

In [ ]:
columnas_a_eliminar = ['city', 'state', 'is_open']

combined_df_business_reviwes = combined_df_business_reviwes.drop(columnas_a_eliminar, axis=1)

In [ ]:
nombres_nuevos = {'estado': 'state', 'ciudad': 'city'}

combined_df_business_reviwes.rename(columns=nombres_nuevos, inplace=True)

In [ ]:
combined_df_business_reviwes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47691 entries, 0 to 47690
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   review_id       47691 non-null  object        
 1   business_id     47691 non-null  object        
 2   stars           47691 non-null  int64         
 3   text            47691 non-null  object        
 4   date            47691 non-null  datetime64[ns]
 5   name            47691 non-null  object        
 6   address         46257 non-null  object        
 7   postal_code     47684 non-null  float64       
 8   latitude        47691 non-null  float64       
 9   longitude       47691 non-null  float64       
 10  business_stars  47691 non-null  float64       
 11  review_count    47691 non-null  int64         
 12  categories      47691 non-null  object        
 13  state           47691 non-null  object        
 14  city            47691 non-null  object        
dtypes:

finalmente, guardamos el dataframe en un archivo csv

In [ ]:
combined_df_business_reviwes.to_csv('/content/drive/My Drive/Colab Notebooks/yelp review business.csv', index=False, encoding='utf-8')